# Key aspects of ECSS

## 1. Creating an ECS scene

Initially, we should load our basic libraries, using the following code.

In [ ]:
import numpy as np

import pyglGA.ECSS.utilities as util
from pyglGA.ECSS.Entity import Entity
from pyglGA.ECSS.Component import BasicTransform, Camera, RenderMesh
from pyglGA.ECSS.System import System, TransformSystem, CameraSystem, RenderSystem
from pyglGA.ext.Scene import Scene
from pyglGA.ECSS.ECSSManager import ECSSManager

from pyglGA.ext.Shader import InitGLShaderSystem, Shader, ShaderGLDecorator, RenderGLShaderSystem
from pyglGA.ext.VertexArray import VertexArray

After deciding the scenegraph in our mind we can easily create the respective tree in pyglGA. For example, suppose we have the following scene in mind:

![scenegraph-example.png](./Files_for_notebooks/scenegraph-example.png)

The code implementing it is the following:

In [ ]:
scene = Scene()    

# Scenegraph with Entities, Components
rootEntity = scene.world.createEntity(Entity(name="Root"))
entityCam1 = scene.world.createEntity(Entity(name="entityCam1"))
scene.world.addEntityChild(rootEntity, entityCam1)
trans1 = scene.world.addComponent(entityCam1, BasicTransform(name="trans1", trs=util.identity()))

entityCam2 = scene.world.createEntity(Entity(name="entityCam2"))
scene.world.addEntityChild(entityCam1, entityCam2)
trans2 = scene.world.addComponent(entityCam2, BasicTransform(name="trans2", trs=util.identity()))
orthoCam = scene.world.addComponent(entityCam2, Camera(util.ortho(-100.0, 100.0, -100.0, 100.0, 1.0, 100.0), "orthoCam","Camera","500"))

node4 = scene.world.createEntity(Entity(name="node4"))
scene.world.addEntityChild(rootEntity, node4)
trans4 = scene.world.addComponent(node4, BasicTransform(name="trans4", trs=util.identity()))
mesh4 = scene.world.addComponent(node4, RenderMesh(name="mesh4"))

Note that we easile created the entities with the `createEntity` command and attached to their respective parent with the `addEntityChild`. The `addComponent` command adds various components (e.g., `BasicTransform`,`Camera`,`RenderMesh`) to the entities.

Let us now create the geometry of the `node4` entity. For example, suppose we want a cube. 

In [ ]:
vertexCube = np.array([
            [-0.5, -0.5, 0.5, 1.0],
            [-0.5, 0.5, 0.5, 1.0],
            [0.5, 0.5, 0.5, 1.0],
            [0.5, -0.5, 0.5, 1.0], 
            [-0.5, -0.5, -0.5, 1.0], 
            [-0.5, 0.5, -0.5, 1.0], 
            [0.5, 0.5, -0.5, 1.0], 
            [0.5, -0.5, -0.5, 1.0]
        ],dtype=np.float32) 
colorCube = np.array([
            [0.0, 0.0, 0.0, 1.0],
            [1.0, 0.0, 0.0, 1.0],
            [1.0, 1.0, 0.0, 1.0],
            [0.0, 1.0, 0.0, 1.0],
            [0.0, 0.0, 1.0, 1.0],
            [1.0, 0.0, 1.0, 1.0],
            [1.0, 1.0, 1.0, 1.0],
            [0.0, 1.0, 1.0, 1.0]
        ], dtype=np.float32)
        

indexCube = np.array((1,0,3, 1,3,2, 
                          2,3,7, 2,7,6,
                          3,0,4, 3,4,7,
                          6,5,1, 6,1,2,
                          4,5,6, 4,6,7,
                          5,4,0, 5,0,1), np.uint32) #rhombus out of two triangles

We shall now add the Systems that parse the Scenegraph. 

In [ ]:
transUpdate = scene.world.createSystem(TransformSystem("transUpdate", "TransformSystem", "001"))
camUpdate = scene.world.createSystem(CameraSystem("camUpdate", "CameraUpdate", "200"))
renderUpdate = scene.world.createSystem(RenderGLShaderSystem())
initUpdate = scene.world.createSystem(InitGLShaderSystem())

In [ ]:
model = util.translate(0.0,0.0,0.5)
eye = util.vec(0.0, 0.0, 5.0)
target = util.vec(0,0,0)
up = util.vec(0.0, 1.0, 0.0)
view = util.lookat(eye, target, up)
#projMat = util.frustum(-10.0, 10.0,-10.0,10.0, -1.0, 10)
projMat = util.perspective(120.0, 1.33, 0.1, 100.0)
# projMat = util.ortho(-100.0, 100.0, -100.0, 100.0, -0.5, 100.0)
#projMat = util.ortho(-5.0, 5.0, -5.0, 5.0, 0.1, 100.0)
#mvpMat = projMat @ view @ model
mvpMat = model @ view @ projMat

# decorated components and systems with sample, default pass-through shader with uniform MVP
shaderDec4 = scene.world.addComponent(node4, ShaderGLDecorator(Shader(vertex_source = Shader.COLOR_VERT_MVP, fragment_source=Shader.COLOR_FRAG)))
shaderDec4.setUniformVariable(key='modelViewProj', value=mvpMat, mat4=True)

# attach a simple cube in a RenderMesh so that VertexArray can pick it up
mesh4.vertex_attributes.append(vertexCube)
mesh4.vertex_attributes.append(colorCube)
mesh4.vertex_index.append(indexCube)
vArray4 = scene.world.addComponent(node4, VertexArray())

# 


We may check that these entities, compononets and systems are correctly registered by running:

In [ ]:
scene.world.print()

## 2 Running the ECSS
__Note:__ You may have to "force quit" the python window or/and the kernel may crash/restart!

In [ ]:
running = True
# MAIN RENDERING LOOP
scene.init(imgui=True, windowWidth = 1024, windowHeight = 768, windowTitle = "pyglGA Cube Scene")

# pre-pass scenegraph to initialise all GL context dependent geometry, shader classes
# needs an active GL context
scene.world.traverse_visit(initUpdate, scene.world.root)

while running:
    running = scene.render(running)
    scene.world.traverse_visit(renderUpdate, scene.world.root)
    scene.render_post()

scene.shutdown()


print("TestScene:test_renderCube END".center(100, '-'))